In [1]:
from pyomo.environ import *
import numpy as np
import pandas as pd

In [2]:
df = pd.read_excel('C://Users//kongl//Desktop//NYU Shanghai//Winter 2022//Optimization Modeling//Data & Excel Files/VerizonPricing_data.xls',sheet_name='Data',header=1)

In [3]:
v = df.drop(columns = "Customer")

In [4]:
v

,Internet,TV,Cell Phone
0,5.25,10.5,5.250
1,26.25,52.5,5.250
2,42.00,42.0,73.500
3,105.00,105.0,0.000
4,0.00,10.5,21.000
...,...,...,...
72,0.00,10.5,0.000
73,52.50,52.5,84.000
74,31.50,42.0,42.000
75,0.00,63.0,10.500


# No Bundling Strategy

In [5]:
Price = range(3)
Bundle = range(3)
Customer = range(77)

In [6]:
model = ConcreteModel()
model.p = Var(Price, within=NonNegativeReals)
model.x = Var(Customer, Bundle, within=Binary)
model.z = Var(Customer, Bundle, within=NonNegativeReals)

In [7]:
model.obj = Objective(
    expr = sum(model.z[i,k] for i in Customer for k in Bundle), 
    sense = maximize)

In [8]:
def no_negative_decision_rule(model,i):
    return sum(v.iloc[i,k]*model.x[i,k]-model.z[i,k] for k in Bundle) >= 0

model.no_negative_decision_con = Constraint(Customer, rule=no_negative_decision_rule)

In [9]:
def optimal_decision_rule(model,i,j):
    return sum(v.iloc[i,k]*model.x[i,k]-model.z[i,k] for k in Bundle) >= v.iloc[i,j] - model.p[j]

model.optimal_decision_con = Constraint(Customer, Bundle, rule=optimal_decision_rule)

In [10]:
def z_zero_rule(model,i,k):
    return model.z[i,k] >= 0

model.z_zero_con = Constraint(Customer, Bundle, rule=z_zero_rule)

In [11]:
def z_price_rule(model,i,k):
    return model.z[i,k] <= model.p[k]
model.z_price_con = Constraint(Customer, Bundle, rule=z_price_rule)

In [12]:
def z_upper_rule(model,i,k):
    return model.z[i,k] <= 10000*model.x[i,k]

model.z_upper_con = Constraint(Customer, Bundle, rule=z_upper_rule)

In [13]:
def z_lower_rule(model,i,k):
    return model.z[i,k] >= model.p[k] - 10000*(1-model.x[i,k])

model.z_lower_con = Constraint(Customer, Bundle, rule=z_lower_rule)

In [14]:
def each_customer_rule(model,i):
    return sum(model.x[i,k] for k in Bundle) <= 3

model.each_customer_con = Constraint(Customer, rule=each_customer_rule)

In [15]:
opt = SolverFactory('gurobi', solver_io="python").solve(model)

In [16]:
df_no = pd.DataFrame()
df_no['Internet'] = v['Internet']
df_no['TV'] = v['TV']
df_no['Cell Phone'] = v['Cell Phone']

In [17]:
for k in range(3):    
    df_no.iloc[:,k] = [value(model.x[i,k]) for i in Customer]

In [18]:
print("The optimal prices for no bundling strategy are as following:")
for k in range(3):
    print("The optimal price for {} is ${:.3f}.".format(v.columns[k],value(model.p[k])))
print("\nWith this price setting, we have:")
for k in range(3):
    print("{} customers purchase {}.".format(sum(df_no.iloc[:,k] == 1),df_no.columns[k]))    
print("The maximized revenue would be ${:.3f}.".format(value(model.obj)))

The optimal prices for no bundling strategy are as following:
The optimal price for Internet is $30.975.
The optimal price for TV is $51.975.
The optimal price for Cell Phone is $51.975.

With this price setting, we have:
46 customers purchase Internet.
50 customers purchase TV.
27 customers purchase Cell Phone.
The maximized revenue would be $5426.925.


# Mixed Bundling Strategy

In [19]:
v_mix = pd.DataFrame()
v_mix['Internet'] = v['Internet']
v_mix['TV'] = v['TV']
v_mix['Cell Phone'] = v['Cell Phone']
v_mix["Internet and TV"] = v_mix["Internet"] + v_mix["TV"]
v_mix["Internet and Cell Phone"] = v_mix["Internet"] + v_mix["Cell Phone"]
v_mix["TV and Cell Phone"] = v_mix["TV"] + v_mix["Cell Phone"]
v_mix["TV,Cell Phone,and Internet"] = v_mix["TV"] + v_mix["Cell Phone"] + v_mix["TV"]

In [20]:
v_mix

,Internet,TV,Cell Phone,Internet and TV,Internet and Cell Phone,TV and Cell Phone,"TV,Cell Phone,and Internet"
0,5.25,10.5,5.250,15.75,10.500,15.750,26.250
1,26.25,52.5,5.250,78.75,31.500,57.750,110.250
2,42.00,42.0,73.500,84.00,115.500,115.500,157.500
3,105.00,105.0,0.000,210.00,105.000,105.000,210.000
4,0.00,10.5,21.000,10.50,21.000,31.500,42.000
...,...,...,...,...,...,...,...
72,0.00,10.5,0.000,10.50,0.000,10.500,21.000
73,52.50,52.5,84.000,105.00,136.500,136.500,189.000
74,31.50,42.0,42.000,73.50,73.500,84.000,126.000
75,0.00,63.0,10.500,63.00,10.500,73.500,136.500


In [21]:
Price = range(7)
Bundle = range(7)
Customer = range(77)

In [22]:
model2 = ConcreteModel()
model2.p = Var(Price, within=NonNegativeReals)
model2.x = Var(Customer, Bundle, within=Binary)
model2.z = Var(Customer, Bundle, within=NonNegativeReals)

In [23]:
model2.obj = Objective(
    expr = sum(model2.z[i,k] for i in Customer for k in Bundle), 
    sense = maximize)

In [24]:
def each_customer_rule2(model2,i):
    return sum(model2.x[i,k] for k in Bundle) <= 1

model2.each_customer_con = Constraint(Customer, rule=each_customer_rule2)

In [25]:
def no_negative_decision_rule2(model2,i):
    return sum(v_mix.iloc[i,k]*model2.x[i,k]-model2.z[i,k] for k in Bundle) >= 0

model2.no_negative_decision_con = Constraint(Customer, rule=no_negative_decision_rule2)

In [26]:
def optimal_decision_rule2(model2,i,j):
    return sum(v_mix.iloc[i,k]*model2.x[i,k]-model2.z[i,k] for k in Bundle) >= v_mix.iloc[i,j] - model2.p[j]

model2.optimal_decision_con = Constraint(Customer, Bundle, rule=optimal_decision_rule2)

In [27]:
def z_zero_rule2(model2,i,k):
    return model2.z[i,k] >= 0

model2.z_zero_con = Constraint(Customer, Bundle, rule=z_zero_rule2)

In [28]:
def z_price_rule2(model2,i,k):
    return model2.z[i,k] <= model2.p[k]
model2.z_price_con = Constraint(Customer, Bundle, rule=z_price_rule2)

In [29]:
def z_upper_rule2(model2,i,k):
    return model2.z[i,k] <= 5000*model2.x[i,k]

model2.z_upper_con = Constraint(Customer, Bundle, rule=z_upper_rule2)

In [30]:
def z_lower_rule2(model2,i,k):
    return model2.z[i,k] >= model2.p[k] - 5000*(1-model2.x[i,k])

model2.z_lower_con = Constraint(Customer, Bundle, rule=z_lower_rule2)

In [31]:
opt = SolverFactory('gurobi', solver_io="python").solve(model2)

In [32]:
df_mix = pd.DataFrame()
df_mix = v_mix.iloc[:,0:7]

In [33]:
for k in range(7):    
    df_mix.iloc[:,k] = [value(model2.x[i,k]) for i in Customer]

In [34]:
print("The optimal prices for mixed bundling strategy are as following:")
for k in range(7):
    print("The optimal price for {} is ${:.3f}.".format(v_mix.columns[k],value(model2.p[k])))
print("\nWith this price setting, we have:")
for k in range(7):
    print("{} customers purchase {}.".format(sum(df_mix.iloc[:,k] == 1),df_mix.columns[k]))
print("The maximized revenue would be ${:.3f}.".format(value(model2.obj)))

The optimal prices for mixed bundling strategy are as following:
The optimal price for Internet is $31.500.
The optimal price for TV is $52.500.
The optimal price for Cell Phone is $42.000.
The optimal price for Internet and TV is $94.500.
The optimal price for Internet and Cell Phone is $84.000.
The optimal price for TV and Cell Phone is $5000.000.
The optimal price for TV,Cell Phone,and Internet is $120.750.

With this price setting, we have:
6 customers purchase Internet.
5 customers purchase TV.
1 customers purchase Cell Phone.
2 customers purchase Internet and TV.
3 customers purchase Internet and Cell Phone.
0 customers purchase TV and Cell Phone.
44 customers purchase TV,Cell Phone,and Internet.
The maximized revenue would be $6247.500.


# Pure Bundling Strategy

In [35]:
v_pure = pd.DataFrame()
v_pure["TV,Cell Phone,and Internet"] = v_mix.loc[:,'TV,Cell Phone,and Internet']

In [36]:
v_pure

,"TV,Cell Phone,and Internet"
0,26.250
1,110.250
2,157.500
3,210.000
4,42.000
...,...
72,21.000
73,189.000
74,126.000
75,136.500


In [37]:
Price = range(1)
Bundle = range(1)
Customer = range(77)

In [38]:
model3 = ConcreteModel()
model3.p = Var(Price, within=NonNegativeReals)
model3.x = Var(Customer, Bundle, within=Binary)
model3.z = Var(Customer, Bundle, within=NonNegativeReals)

In [39]:
model3.obj = Objective(
    expr = sum(model3.z[i,k] for i in Customer for k in Bundle), 
    sense = maximize)

In [40]:
def no_negative_decision_rule3(model3,i):
    return sum(v_pure.iloc[i,k]*model3.x[i,k]-model3.z[i,k] for k in Bundle) >= 0

model3.no_negative_decision_con = Constraint(Customer, rule=no_negative_decision_rule3)

In [41]:
def optimal_decision_rule3(model3,i,j):
    return sum(v_pure.iloc[i,k]*model3.x[i,k]-model3.z[i,k] for k in Bundle) >= v_pure.iloc[i,j] - model3.p[j]

model3.optimal_decision_con = Constraint(Customer, Bundle, rule=optimal_decision_rule3)

In [42]:
def z_zero_rule3(model3,i,k):
    return model3.z[i,k] >= 0

model3.z_zero_con = Constraint(Customer, Bundle, rule=z_zero_rule3)

In [43]:
def z_price_rule3(model3,i,k):
    return model3.z[i,k] <= model3.p[k]
model3.z_price_con = Constraint(Customer, Bundle, rule=z_price_rule3)

In [44]:
def z_upper_rule3(model3,i,k):
    return model3.z[i,k] <= 10000*model3.x[i,k]

model3.z_upper_con = Constraint(Customer, Bundle, rule=z_upper_rule3)

In [45]:
def z_lower_rule3(model3,i,k):
    return model3.z[i,k] >= model3.p[k] - 10000*(1-model3.x[i,k])

model3.z_lower_con = Constraint(Customer, Bundle, rule=z_lower_rule3)

In [46]:
opt = SolverFactory('gurobi', solver_io="python").solve(model3)

In [47]:
print("The optimal price for pure bundling strategy is ${:.3f}.".format(value(model3.p[0])))
print("With this price setting, we have {:.0f} customers purchasing the pure bundle.".format(sum([value(model.x[i,0]) for i in range(77)])))
print("The maximized revenue would be ${:.3f}.".format(value(model3.obj)))

The optimal price for pure bundling strategy is $120.750.
With this price setting, we have 46 customers purchasing the pure bundle.
The maximized revenue would be $5675.250.
